In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#使用的是3-3(4-1) MNIST数据集（手写数据）分类简单版本-二次代价函数改为对数释然代价函数暨4-1交叉熵
#暨 04 MNIST数据集（手写数据）分类简单版本-二次代价函数改为对数释然代价函数 交叉熵




#載入數據集，one_hot是把数据转化为只有0和1的形式
mnist = input_data.read_data_sets("MNIST_data", one_hot = True) 

#會將資料集分批次（一次100张）放入神经网络进行训练，
#每一個批次的大小
batch_size = 100 

#計算一共有多少批次，训练集数量mnist.train.num_examples 
# // 在python中表示取商
n_batch = mnist.train.num_examples // batch_size      

#命名空间
with tf.name_scope('input'):
    #定义兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
    x = tf.placeholder(tf.float32, [None, 784], name='x-input') # 28 * 28 = 784，None值变为100
    y = tf.placeholder(tf.float32, [None, 10], name='y-input') #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量

#创建一個簡單的神經網路 (只有输入层和輸出層，输入层784个神经元，输出层總共10個神經元，即十个标签)
W = tf.Variable(tf.zeros([784, 10]))              #权值
b = tf.Variable(tf.zeros([10]))  
prediction = tf.nn.softmax(tf.matmul(x, W) + b)   #预测值，用到softmax


#二次代價函數 : loss = mean((y - prediction)^2)
#loss = tf.reduce_mean(tf.square(y - prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction)) #交叉熵

#使用梯度下降法
#Gradient desent method  (learning rate = 0.2)
#最小化 代價函數 (operator)
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
#以上两句可以合并为 train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量 operator
init = tf.global_variables_initializer()



#测试训练的准确率，求准确率的方法
#如果y標籤最大的值，與prediction標籤最大的值相等，則回傳true
#結果存在一個 boolean 的变量correct_prediction中

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
                              
 #argmax 返回一維張量中最大的值所在的位置
 # 求标签y里面最大的值在哪个位置即标签
 # tf.argmax(prediction, 1)预测 概率最大就会判定识别的这张图片是属于哪个标签的
 # (tf.argmax(y, 1)，真实样本的y存放的都是0或1，哪位是1就会返回哪位的值
 #  然后再比较上面两者，是否一样
                             
                              
# 求准确率
# 轉換資料格式 boolean 轉成 32位的float，接著再取平均值，得到准确率
# true转换为1.0，false转换为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         
                              
#開始training
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph) #文件夹存放的是sess.graph图的结构
    for epoch in range(1): #总共疊代1次，因为主要是使用tensorboard来画神经网络结构的
        for batch in range(n_batch):#每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs图片数据, batch_ys图片标签
            # mnist.train.next_batch(batch_size)是获取下一个一百张图片               
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)               
            sess.run(train_step, feed_dict = {x: batch_xs, y: batch_ys})
                              
        #每做完一次 outer loop 計算一次准确率
        acc = sess.run(accuracy, feed_dict = {x: mnist.test.images, y: mnist.test.labels})
                   
        # str(epoch)周期数
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter =0, Testing Accuracy =0.8242
